In [1]:
from torchvision import transforms
from torchvision import models
import torch.backends.cudnn as cudnn
import torch.nn as nn
import pandas as pd
import torch
import os
from dataset_preprocessing import Paths, Dataset
from utils import CROPP_FUNCS, TRAIN_PARTS

/home/noonmare/anaconda3/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/noonmare/anaconda3/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
  warn(f"Failed to load image Python extension: {e}")


In [ ]:
ds = Dataset(path_to_the_dataset=Paths.pandora_18k)

In [4]:
cudnn.benchmark = False

HEIGHT = WIDTH = 299

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

img_transform = transforms.Compose([
    transforms.Resize((WIDTH, HEIGHT)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

model_path = Paths.pandora_18k + 'Conv_models/Inception-V3/inc_v3_model.pth'
model_path_lu = Paths.pandora_18k + 'Conv_models/Inception-V3/inc_v3_model_lu.pth'
model_path_ru = Paths.pandora_18k + 'Conv_models/Inception-V3/inc_v3_model_ru.pth'
model_path_c = Paths.pandora_18k + 'Conv_models/Inception-V3/inc_v3_model_c.pth'
model_path_ld = Paths.pandora_18k + 'Conv_models/Inception-V3/inc_v3_model_ld.pth'
model_path_rd = Paths.pandora_18k + 'Conv_models/Inception-V3/inc_v3_model_rd.pth'

conv_nn = models.inception_v3(weights='IMAGENET1K_V1')
num_features = conv_nn.fc.in_features
conv_nn.fc = nn.Linear(num_features, len(ds.classes))
conv_nn.aux_logits=False                                                                                                                                                                                                                                                                                                                                        
conv_nn.load_state_dict(torch.load(model_path))
conv_nn.eval()

conv_nn.to(device)

conv_nn_lu = models.inception_v3(weights='IMAGENET1K_V1')
num_features = conv_nn_lu.fc.in_features
conv_nn_lu.fc = nn.Linear(num_features, len(ds.classes))
conv_nn_lu.aux_logits=False                                                                                                                                                                                                                                                                                                                                        
conv_nn_lu.load_state_dict(torch.load(model_path_lu))
conv_nn_lu.eval()

conv_nn_lu.to(device)

conv_nn_ru = models.inception_v3(weights='IMAGENET1K_V1')
num_features = conv_nn_ru.fc.in_features
conv_nn_ru.fc = nn.Linear(num_features, len(ds.classes))
conv_nn_ru.aux_logits=False                                                                                                                                                                                                                                                                                                                                        
conv_nn_ru.load_state_dict(torch.load(model_path_ru))
conv_nn_ru.eval()

conv_nn_ru.to(device)

conv_nn_c = models.inception_v3(weights='IMAGENET1K_V1')
num_features = conv_nn_c.fc.in_features
conv_nn_c.fc = nn.Linear(num_features, len(ds.classes))
conv_nn_c.aux_logits=False                                                                                                                                                                                                                                                                                                                                        
conv_nn_c.load_state_dict(torch.load(model_path_c))
conv_nn_c.eval()

conv_nn_c.to(device)

conv_nn_ld = models.inception_v3(weights='IMAGENET1K_V1')
num_features = conv_nn_ld.fc.in_features
conv_nn_ld.fc = nn.Linear(num_features, len(ds.classes))
conv_nn_ld.aux_logits=False                                                                                                                                                                                                                                                                                                                                        
conv_nn_ld.load_state_dict(torch.load(model_path_ld))
conv_nn_ld.eval()

conv_nn_ld.to(device)

conv_nn_rd = models.inception_v3(weights='IMAGENET1K_V1')
num_features = conv_nn_rd.fc.in_features
conv_nn_rd.fc = nn.Linear(num_features, len(ds.classes))
conv_nn_rd.aux_logits=False                                                                                                                                                                                                                                                                                                                                        
conv_nn_rd.load_state_dict(torch.load(model_path_rd))
conv_nn_rd.eval()

conv_nn_rd.to(device)

cropp_funcs_cnns = [(CROPP_FUNCS["lu"], conv_nn_lu), (CROPP_FUNCS["ru"], conv_nn_ru), (CROPP_FUNCS["ld"], conv_nn_ld), (CROPP_FUNCS["rd"], conv_nn_rd), (CROPP_FUNCS["c"], conv_nn_c), (lambda im : Image.open(im).convert("RGB"), conv_nn)]

columns = []

parts = ["lu", "ru", "ld", "rd", "c", "full"]

for p in parts:
    for c in ds.classes:
        columns.append(c + '_' + p)

columns.append("label")

train_emb = pd.DataFrame(columns=columns)
valid_emb = pd.DataFrame(columns=columns)
test_emb = pd.DataFrame(columns=columns)

In [5]:
from functools import reduce

softmax = torch.nn.Softmax(dim=1)

for tp in TRAIN_PARTS:
    path = Paths.pandora_18k + tp + '/'
    for ind, cl in enumerate(ds.classes):
        cl_path = path + cl + '/'
        cl_imgs = os.listdir(cl_path)
        for img_name in cl_imgs:
            img_path = cl_path + img_name
            parts_probs = []
            for cr_func, cnn in cropp_funcs_cnns:
                timg = cr_func(img_path)
                timg = torch.unsqueeze(img_transform(timg), 0).to(device)
                output = cnn(timg)
                probs = softmax(output)
                parts_probs.append(probs)   
            prob_emb = reduce(lambda x, y: torch.cat([x, y], axis=1), parts_probs).tolist()[0] + [ind+1]
            if tp == "Train":
                train_emb.loc[len(train_emb)] = prob_emb
            if tp == "Validation":
                valid_emb.loc[len(valid_emb)] = prob_emb
            if tp == "Test":
                test_emb.loc[len(test_emb)] = prob_emb

In [6]:
new_columns = []

for c in ds.classes:
    for p in parts:
        new_columns.append(c + '_' + p)

new_columns.append("label")

train_emb = train_emb[new_columns]
valid_emb = valid_emb[new_columns]
test_emb = test_emb[new_columns]

In [9]:
inc_v3_path =  Paths.pandora_18k + 'Conv_models/Inception-V3/'

train_emb_path = inc_v3_path + 'train_full_emb.csv'
valid_emb_path = inc_v3_path + 'valid_full_emb.csv'
test_emb_path = inc_v3_path + 'test_full_emb.csv'

train_emb.to_csv(train_emb_path, index=False)
valid_emb.to_csv(valid_emb_path, index=False)
test_emb.to_csv(test_emb_path, index=False)